In [2]:
import numpy as np
import matplotlib.pyplot as plt
from idanalysis.analysis import Tools
from idanalysis import IDKickMap
import utils
from fieldmaptrack import Beam
from mathphys.functions import save_pickle, load_pickle


In [15]:
def get_sw_fname(phase, dgv):

    ph1 = '/opt/lnls-ima/delta52/id-sabia/model-03/measurement/magnetic/stretchedwire/delta_sabia_final/Delta_Sabia_Final_GV=26.25_PH=-13.125_231029_1618.dat'

    ph2 = '/opt/lnls-ima/delta52/id-sabia/model-03/measurement/magnetic/stretchedwire/delta_sabia_final/Delta_Sabia_Final_GV=-6.562_PH=13.125_231029_1217.dat'

    ph3 = '/opt/lnls-ima/delta52/id-sabia/model-03/measurement/magnetic/stretchedwire/delta_sabia_final/Delta_Sabia_Final_GV=0_PH=-6.562_231029_2005.dat'

    ph4 = '/opt/lnls-ima/delta52/id-sabia/model-03/measurement/magnetic/stretchedwire/delta_sabia_final/Delta_Sabia_Final_GV=0_PH=6.562_231030_1017.dat'


    data_dict = {
            (-13.125, 26.25): ph1,
            (13.125, 6.5625): ph2,
            (-6.5625, 0): ph3,
            (6.5625, 0): ph4,
        }
    return data_dict[(phase, dgv)]

def get_field_integrals(phase, dgv):
    fname = get_sw_fname(phase, dgv)
    file = open(fname, 'r')
    x = list()
    I1x = list()
    I1y = list()
    I2x = list()
    I2y = list()
    for i, line in enumerate(file):
        columns= line.split('\t')
        if i > 0:
            x.append(float(columns[0]))
            I1x.append(float(columns[2]))
            I1y.append(float(columns[4]))
            I2x.append(float(columns[5]))
            I2y.append(float(columns[7]))

    # Closing files
    file.close()
    x = np.array(x)
    I1x = np.array(I1x)
    I1y = np.array(I1y)
    I2x = np.array(I2x)
    I2y = np.array(I2y)
    if phase != -13.125:
        return x, I1x, I1y, I2x, I2y

    x = x.reshape(3, 11)
    I1x = I1x.reshape(3, 11)
    I2x = I2x.reshape(3, 11)
    I1y = I1y.reshape(3, 11)
    I2y = I2y.reshape(3, 11)
    return x[1, :], I1x[1, :], I1y[1, :], I2x[1, :], I2y[1, :]


In [29]:
def get_kmap_fname(phase, dgv):


    ph1 = '/opt/lnls-ima/delta52/id-sabia/model-03/measurement/magnetic/hallprobe/delta_sabia_final/CircN/kickmaps/2023-10-26_DeltaSabia_Phase=-13.125mm_GV=26.25mm_Fieldmap_Corrected_X=-5_5mm_Z=-900_900mm_Y=0mm_ID=4871.kck'

    ph2 = '/opt/lnls-ima/delta52/id-sabia/model-03/measurement/magnetic/hallprobe/delta_sabia_final/CircP/kickmaps/2023-10-27_DeltaSabia_Phase=13.125mm_GV=-6.5625mm_Fieldmap_Corrected_X=-5_5mm_Z=-900_900mm_Y=0mm_ID=4878.kck'

    ph3 = '/opt/lnls-ima/delta52/id-sabia/model-03/measurement/magnetic/hallprobe/delta_sabia_final/zeroK/kickmaps/2023-10-27_DeltaSabia_Phase=-6.5625mm_GV=0mm_Fieldmap_Corrected_X=-5_5mm_Z=-900_900mm_Y=0mm_ID=4882.kck'

    ph4 = '/opt/lnls-ima/delta52/id-sabia/model-03/measurement/magnetic/hallprobe/delta_sabia_final/zeroK/kickmaps/2023-10-27_DeltaSabia_Phase=6.5625mm_GV=0mm_Fieldmap_Corrected_X=-5_5mm_Z=-900_900mm_Y=0mm_ID=4884.kck'


    data_dict = {
            (-13.125, 26.25): ph1,
            (13.125, 6.5625): ph2,
            (-6.5625, 0): ph3,
            (6.5625, 0): ph4,
        }
    return data_dict[(phase, dgv)]

def get_kickmap_kicks(phase, dgv):
    kmap_fname = get_kmap_fname(phase, dgv)
    idkickmap = IDKickMap(kmap_fname=kmap_fname)

    return idkickmap.posx, idkickmap.kicky[0, :], idkickmap.kickx[0, :]


In [23]:
def i1_2_kick(i1, brho):
    kick = -1*i1 *1e-6
    return kick/brho

def i2_2_pos(i2, brho):
    pos = -1*i2*1e-5
    return pos/brho


In [59]:
beam = Beam(energy=3)
brho = beam.brho

phase = -6.5625
dgv = 0

x, i1x, i1y, i2x, i2y = get_field_integrals(phase, dgv)
posx, kicky_kickmap, kickx_kickmap = get_kickmap_kicks(phase, dgv)
posy = np.array([-0.0025, 0, 0.0025])

py_sw = i1_2_kick(i1x, brho)
px_sw = i1_2_kick(i1y, brho)
ry_sw = i2_2_pos(i2x, brho)
rx_sw = i2_2_pos(i2y, brho)
kicky_sw = py_sw * brho**2

kickx = np.zeros((len(posy), len(posx)))
kicky = np.zeros((len(posy), len(posx)))
rx = np.zeros((len(posy), len(posx)))
ry = np.zeros((len(posy), len(posx)))

for i, y in enumerate(posy):
    kickx[i, :] = kickx_kickmap
    kicky[i, :] = kicky_sw
    rx[i, :] = rx_sw
    ry[i, :] = ry_sw

idkickmap = IDKickMap()
idkickmap.posx = posx
idkickmap.posy = posy
idkickmap.kickx = kickx
idkickmap.kicky = kicky
idkickmap.fposx = rx
idkickmap.fposy = ry

phase_str = Tools.get_phase_str(phase)
fname = 'kickmap-ID_phase{}_dgv{}_fix_True.txt'.format(phase_str, dgv)
idkickmap.save_kickmap_file(fname)
